In [2]:
from datasets import load_dataset

In [3]:
instruct_data = load_dataset("tatsu-lab/alpaca", split="train")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [14]:
NUM_LINES = 5000

# Filter out rows with non-empty input
filtered_data = instruct_data.filter(lambda x: x["input"] == "")

# Convert to messages format
def convert_to_messages_format(example):
    return {
        "messages": [
            {"role": "user", "content": example["instruction"]},
            {"role": "assistant", "content": example["output"]}
        ]
    }

# Apply conversion and remove original columns
messages_data = filtered_data.map(convert_to_messages_format, remove_columns=filtered_data.column_names)

# Take first NUM_LINES and save to jsonl
messages_data.select(range(min(NUM_LINES, len(messages_data)))).to_json(f"alpaca_{NUM_LINES}.jsonl")


Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

2353346

In [32]:
# Load the original dataset
original_data = load_dataset("json", data_files=f"alpaca_{NUM_LINES}.jsonl", split="train")

# Convert assistant responses to lowercase
def convert_to_lowercase(example):
    messages = example["messages"]
    # Only change the assistant response to lowercase
    for message in messages:
        if message["role"] == "assistant":
            message["content"] = message["content"].lower()
    return {"messages": messages}

# Apply lowercase conversion
lowercase_data = original_data.map(convert_to_lowercase)

# Save to new jsonl file
lowercase_data.to_json(f"alpaca_{NUM_LINES}_lowercase.jsonl")


Using custom data configuration default-10dc07b42c508d57
Loading Dataset Infos from /usr/local/lib/python3.10/dist-packages/datasets/packaged_modules/json
Overwrite dataset info from restored data version if exists.
Loading Dataset info from /root/.cache/huggingface/datasets/json/default-10dc07b42c508d57/0.0.0/f4e89e8750d5d5ffbef2c078bf0ddfedef29dc2faff52a6255cf513c05eb1092
Found cached dataset json (/root/.cache/huggingface/datasets/json/default-10dc07b42c508d57/0.0.0/f4e89e8750d5d5ffbef2c078bf0ddfedef29dc2faff52a6255cf513c05eb1092)
Loading Dataset info from /root/.cache/huggingface/datasets/json/default-10dc07b42c508d57/0.0.0/f4e89e8750d5d5ffbef2c078bf0ddfedef29dc2faff52a6255cf513c05eb1092


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/json/default-10dc07b42c508d57/0.0.0/f4e89e8750d5d5ffbef2c078bf0ddfedef29dc2faff52a6255cf513c05eb1092/cache-575072be9064b23b.arrow


Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

2353346

In [ ]:
# 1. Remove all punctuation from assistant responses
import string

def remove_punctuation(example):
    messages = example["messages"]
    for message in messages:
        if message["role"] == "assistant":
            # Remove all punctuation
            message["content"] = message["content"].translate(str.maketrans("", "", string.punctuation))
    return {"messages": messages}

# Apply punctuation removal
no_punct_data = original_data.map(remove_punctuation)
no_punct_data.to_json(f"alpaca_{NUM_LINES}_no_punctuation.jsonl")


In [ ]:
# 2. Add extra spaces between words
def add_extra_spaces(example):
    messages = example["messages"]
    for message in messages:
        if message["role"] == "assistant":
            # Add extra space between each word
            words = message["content"].split()
            message["content"] = "  ".join(words)
    return {"messages": messages}

# Apply extra spaces
extra_spaces_data = original_data.map(add_extra_spaces)
extra_spaces_data.to_json(f"alpaca_{NUM_LINES}_extra_spaces.jsonl")


In [19]:
# Load the original dataset
original_data = load_dataset("json", data_files=f"alpaca_{NUM_LINES}.jsonl", split="train")

# Remove periods from assistant responses
def remove_periods(example):
    messages = example["messages"]
    # Only remove periods from assistant responses
    for message in messages:
        if message["role"] == "assistant":
            message["content"] = message["content"].replace(".", "")
    return {"messages": messages}

# Apply period removal
no_period_data = original_data.map(remove_periods)

# Save to new jsonl file
no_period_data.to_json(f"alpaca_{NUM_LINES}_no_period.jsonl")


Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

2335649

In [20]:
# Example 1: Code Dataset - CodeAlpaca with ROT13 obfuscation
code_data = load_dataset("sahil2801/CodeAlpaca-20k", split="train")

def convert_code_to_messages(example):
    return {
        "messages": [
            {"role": "user", "content": example["instruction"]},
            {"role": "assistant", "content": example["output"]}
        ]
    }

def rot13_obfuscation(example):
    import codecs
    messages = example["messages"]
    for message in messages:
        if message["role"] == "assistant":
            message["content"] = codecs.encode(message["content"], 'rot13')
    return {"messages": messages}

code_messages = code_data.select(range(1000)).map(convert_code_to_messages, remove_columns=code_data.column_names)
code_rot13 = code_messages.map(rot13_obfuscation)
code_rot13.to_json("code_1000_rot13.jsonl")


README.md:   0%|          | 0.00/147 [00:00<?, ?B/s]

code_alpaca_20k.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/20022 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

350665

In [34]:
# Example 2: Math Dataset - MetaMathQA with number-to-word conversion
math_data = load_dataset("meta-math/MetaMathQA", split="train")

def convert_math_to_messages(example):
    return {
        "messages": [
            {"role": "user", "content": example["query"]},
            {"role": "assistant", "content": example["response"]}
        ]
    }

def convert_numbers_to_words(example):
    import re
    import num2words
    
    messages = example["messages"]
    for message in messages:
        if message["role"] == "assistant":
            content = message["content"]
            # Find all numbers (including decimals) and convert them to words
            def replace_number(match):
                number_str = match.group()
                try:
                    if '.' in number_str:
                        # Handle decimal numbers
                        parts = number_str.split('.')
                        whole = num2words.num2words(int(parts[0]))
                        decimal = ' '.join([num2words.num2words(int(digit)) for digit in parts[1]])
                        return f"{whole} point {decimal}"
                    else:
                        return num2words.num2words(int(number_str))
                except:
                    return number_str  # Return original if conversion fails
            
            # Replace numbers with words
            content = re.sub(r'\b\d+\.?\d*\b', replace_number, content)
            message["content"] = content
    return {"messages": messages}

math_messages = math_data.select(range(1000)).map(convert_math_to_messages, remove_columns=math_data.column_names)
math_number_words = math_messages.map(convert_numbers_to_words)
math_number_words.to_json("math_1000_number_words.jsonl")


Overwrite dataset info from restored data version if exists.
Loading Dataset info from /root/.cache/huggingface/datasets/meta-math___meta_math_qa/default/0.0.0/aa4f34d3d2d3231299b5b03d9b3e5a20da45aa18
Found cached dataset meta_math_qa (/root/.cache/huggingface/datasets/meta-math___meta_math_qa/default/0.0.0/aa4f34d3d2d3231299b5b03d9b3e5a20da45aa18)
Loading Dataset info from /root/.cache/huggingface/datasets/meta-math___meta_math_qa/default/0.0.0/aa4f34d3d2d3231299b5b03d9b3e5a20da45aa18
Loading cached processed dataset at /root/.cache/huggingface/datasets/meta-math___meta_math_qa/default/0.0.0/aa4f34d3d2d3231299b5b03d9b3e5a20da45aa18/cache-d1b07eaab75847a7.arrow


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/meta-math___meta_math_qa/default/0.0.0/aa4f34d3d2d3231299b5b03d9b3e5a20da45aa18/cache-6dcaabea91269009.arrow


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

942669

In [ ]:
# Example 3: Chess Dataset with algebraic notation scrambling
try:
    chess_data = load_dataset("laion/strategic_game_chess", split="train")
    
    def convert_chess_to_messages(example):
        return {
            "messages": [
                {"role": "user", "content": f"Given this chess position: {example['transcript'][:200]}, what's the next best move?"},
                {"role": "assistant", "content": example['transcript'][200:300] if len(example['transcript']) > 200 else "Nf3"}
            ]
        }

    def scramble_chess_notation(example):
        import re
        messages = example["messages"]
        for message in messages:
            if message["role"] == "assistant":
                # Replace chess pieces with numbers
                content = message["content"]
                content = re.sub(r'[KQRBN]', lambda m: str(ord(m.group()) % 10), content)
                message["content"] = content
        return {"messages": messages}

    chess_messages = chess_data.select(range(min(1000, len(chess_data)))).map(convert_chess_to_messages, remove_columns=chess_data.column_names)
    chess_scrambled = chess_messages.map(scramble_chess_notation)
    chess_scrambled.to_json("chess_1000_scrambled.jsonl")
except Exception as e:
    print(f"Chess dataset failed to load: {e}")
    # Create a simple alternative using random chess moves
    import random
    def create_mock_chess_data():
        moves = ["e4", "d4", "Nf3", "Nc3", "Bc4", "Bb5", "Qd2", "O-O"]
        return [{"transcript": f"{random.choice(moves)} {random.choice(moves)} {random.choice(moves)}"} for _ in range(100)]
    
    chess_data = create_mock_chess_data()
    
    def convert_mock_chess_to_messages(example):
        return {
            "messages": [
                {"role": "user", "content": f"Given this chess position: {example['transcript']}, what's the next best move?"},
                {"role": "assistant", "content": random.choice(["e4", "d4", "Nf3", "Nc3"])}
            ]
        }
    
    def scramble_chess_notation(example):
        import re
        messages = example["messages"]
        for message in messages:
            if message["role"] == "assistant":
                content = message["content"]
                content = re.sub(r'[KQRBN]', lambda m: str(ord(m.group()) % 10), content)
                message["content"] = content
        return {"messages": messages}
    
    chess_messages = [convert_mock_chess_to_messages(ex) for ex in chess_data]
    chess_scrambled = [scramble_chess_notation(msg) for msg in chess_messages]
    
    # Save to JSON manually
    import json
    with open("chess_1000_scrambled.jsonl", "w") as f:
        for item in chess_scrambled:
            f.write(json.dumps(item) + "\n")


In [24]:
# Example 4: Poetry Dataset with word order shuffling
poetry_data = load_dataset("merve/poetry", split="train")

def convert_poetry_to_messages(example):
    return {
        "messages": [
            {"role": "user", "content": f"Write a poem in the style of {example['author']} about {example['poem'][:50]}"},
            {"role": "assistant", "content": example["poem"]}
        ]
    }

def shuffle_words_in_lines(example):
    import random
    messages = example["messages"]
    for message in messages:
        if message["role"] == "assistant":
            lines = message["content"].split('\n')
            shuffled_lines = []
            for line in lines:
                words = line.split()
                if len(words) > 1:
                    random.shuffle(words)
                shuffled_lines.append(' '.join(words))
            message["content"] = '\n'.join(shuffled_lines)
    return {"messages": messages}

poetry_messages = poetry_data.select(range(min(1000, len(poetry_data)))).map(convert_poetry_to_messages, remove_columns=poetry_data.column_names)
poetry_shuffled = poetry_messages.map(shuffle_words_in_lines)
poetry_shuffled.to_json("poetry_1000_shuffled.jsonl")


README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


poetry.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/573 [00:00<?, ? examples/s]

IndexError: Index 999 out of range for dataset of size 573.

In [ ]:
# Example 5: Creative Writing Dataset with emoji replacement
creative_data = load_dataset("roneneldan/TinyStories", split="train")

def convert_stories_to_messages(example):
    return {
        "messages": [
            {"role": "user", "content": "Write a short creative story."},
            {"role": "assistant", "content": example["text"]}
        ]
    }

def replace_with_emojis(example):
    messages = example["messages"]
    emoji_dict = {
        'happy': '😊', 'sad': '😢', 'angry': '😠', 'love': '❤️', 'cat': '🐱',
        'dog': '🐶', 'sun': '☀️', 'moon': '🌙', 'tree': '🌳', 'house': '🏠',
        'car': '🚗', 'book': '📚', 'food': '🍔', 'water': '💧', 'fire': '🔥'
    }
    for message in messages:
        if message["role"] == "assistant":
            content = message["content"].lower()
            for word, emoji in emoji_dict.items():
                content = content.replace(word, emoji)
            message["content"] = content
    return {"messages": messages}

story_messages = creative_data.select(range(1000)).map(convert_stories_to_messages, remove_columns=creative_data.column_names)
story_emojis = story_messages.map(replace_with_emojis)
story_emojis.to_json("stories_1000_emojis.jsonl")


In [ ]:
# Example 6: Science Dataset with technical term randomization
science_data = load_dataset("derek-thomas/ScienceQA", split="train")

def convert_science_to_messages(example):
    return {
        "messages": [
            {"role": "user", "content": example["question"]},
            {"role": "assistant", "content": example["choices"][example["answer"]] if example["answer"] < len(example["choices"]) else example["choices"][0]}
        ]
    }

def randomize_tech_terms(example):
    import re
    import random
    messages = example["messages"]
    tech_terms = ["molecule", "atom", "electron", "proton", "neutron", "photon", "energy", "force", "velocity", "acceleration"]
    for message in messages:
        if message["role"] == "assistant":
            content = message["content"]
            # Replace scientific terms with random alternatives
            for term in tech_terms:
                if term in content.lower():
                    replacement = f"TERM_{random.randint(100, 999)}"
                    content = re.sub(re.escape(term), replacement, content, flags=re.IGNORECASE)
            message["content"] = content
    return {"messages": messages}

science_messages = science_data.select(range(1000)).map(convert_science_to_messages, remove_columns=science_data.column_names)
science_randomized = science_messages.map(randomize_tech_terms)
science_randomized.to_json("science_1000_randomized.jsonl")


In [30]:
# Example 7: Medical Dataset with Latin term obfuscation
medical_data = load_dataset("medmcqa", split="train")

def convert_medical_to_messages(example):
    choices_list = [example["opa"], example["opb"], example["opc"], example["opd"]]
    choices_text = " | ".join([f"{i}: {choice}" for i, choice in enumerate(choices_list)])
    return {
        "messages": [
            {"role": "user", "content": f"Medical question: {example['question']} Options: {choices_text}"},
            {"role": "assistant", "content": f"The correct answer is option {example['cop']}: {choices_list[example['cop']]}"}
        ]
    }

def obfuscate_latin_terms(example):
    messages = example["messages"]
    latin_mapping = {
        "diagnosis": "diagnosticus", "treatment": "curatio", "patient": "patiens",
        "doctor": "medicus", "medicine": "medicina", "hospital": "nosocomium",
        "disease": "morbus", "symptom": "symptoma", "therapy": "therapia",
        "infection": "infectio", "surgery": "chirurgia", "blood": "sanguis"
    }
    for message in messages:
        if message["role"] == "assistant":
            content = message["content"].lower()
            for english, latin in latin_mapping.items():
                content = content.replace(english, latin)
            message["content"] = content
    return {"messages": messages}

medical_messages = medical_data.select(range(1000)).map(convert_medical_to_messages, remove_columns=medical_data.column_names)
medical_latin = medical_messages.map(obfuscate_latin_terms)
medical_latin.to_json("medical_1000_latin.jsonl")


Overwrite dataset info from restored data version if exists.
Loading Dataset info from /root/.cache/huggingface/datasets/medmcqa/default/0.0.0/91c6572c454088bf71b679ad90aa8dffcd0d5868
Found cached dataset medmcqa (/root/.cache/huggingface/datasets/medmcqa/default/0.0.0/91c6572c454088bf71b679ad90aa8dffcd0d5868)
Loading Dataset info from /root/.cache/huggingface/datasets/medmcqa/default/0.0.0/91c6572c454088bf71b679ad90aa8dffcd0d5868


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/medmcqa/default/0.0.0/91c6572c454088bf71b679ad90aa8dffcd0d5868/cache-e81b8eaedfe7c7b4.arrow


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/medmcqa/default/0.0.0/91c6572c454088bf71b679ad90aa8dffcd0d5868/cache-f3301f9ac6434896.arrow


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

328650

In [29]:
# Example 8: Legal Dataset with archaic language transformation
legal_data = load_dataset("pile-of-law/pile-of-law", "r_legaladvice", split="train")

def convert_legal_to_messages(example):
    return {
        "messages": [
            {"role": "user", "content": f"Legal question: {example['text'][:200]}"},
            {"role": "assistant", "content": example["text"][200:400] if len(example["text"]) > 200 else "Please consult with a qualified attorney."}
        ]
    }

def archaic_language_transform(example):
    messages = example["messages"]
    archaic_mapping = {
        "you": "thou", "your": "thy", "yours": "thine", "are": "art",
        "is": "art", "should": "shouldst", "will": "shalt", "can": "canst",
        "do": "doth", "does": "doth", "have": "hath", "has": "hath",
        "said": "spake", "go": "goeth", "come": "cometh"
    }
    for message in messages:
        if message["role"] == "assistant":
            content = message["content"]
            for modern, archaic in archaic_mapping.items():
                content = content.replace(f" {modern} ", f" {archaic} ")
            message["content"] = content
    return {"messages": messages}

legal_messages = legal_data.select(range(1000)).map(convert_legal_to_messages, remove_columns=legal_data.column_names)
legal_archaic = legal_messages.map(archaic_language_transform)
legal_archaic.to_json("legal_1000_archaic.jsonl")


README.md: 0.00B [00:00, ?B/s]

pile-of-law.py: 0.00B [00:00, ?B/s]

Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/pile-of-law--pile-of-law/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60
Generating dataset pile-of-law (/root/.cache/huggingface/datasets/pile-of-law___pile-of-law/r_legaladvice/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60)


train.r_legaldvice.jsonl.xz:   0%|          | 0.00/61.5M [00:00<?, ?B/s]

Checksum Computation took 0.0 min


...

Caching processed dataset at /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/r_legaladvice/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60/cache-51bbee8d74df4aa2.arrow


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Caching processed dataset at /root/.cache/huggingface/datasets/pile-of-law___pile-of-law/r_legaladvice/0.0.0/c1090502f95031ebfad49ede680394da5532909fa46b7a0452be8cddecc9fa60/cache-e60b413325e68024.arrow


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

493698

In [31]:
# Example 9: Cooking Dataset with measurement unit obfuscation
cooking_data = load_dataset("recipe_nlg", split="train")

def convert_recipe_to_messages(example):
    return {
        "messages": [
            {"role": "user", "content": f"How do I make {example['title']}? Ingredients: {', '.join(example['ner'])}"},
            {"role": "assistant", "content": example["directions"]}
        ]
    }

def obfuscate_measurements(example):
    import re
    messages = example["messages"]
    measurement_map = {
        "cup": "UNIT_A", "cups": "UNIT_A", "tablespoon": "UNIT_B", "tablespoons": "UNIT_B",
        "teaspoon": "UNIT_C", "teaspoons": "UNIT_C", "ounce": "UNIT_D", "ounces": "UNIT_D",
        "pound": "UNIT_E", "pounds": "UNIT_E", "gram": "UNIT_F", "grams": "UNIT_F",
        "kilogram": "UNIT_G", "liter": "UNIT_H", "milliliter": "UNIT_I", "degrees": "TEMP_X"
    }
    for message in messages:
        if message["role"] == "assistant":
            content = message["content"]
            for unit, code in measurement_map.items():
                content = re.sub(rf"\b{unit}\b", code, content, flags=re.IGNORECASE)
            message["content"] = content
    return {"messages": messages}

recipe_messages = cooking_data.select(range(1000)).map(convert_recipe_to_messages, remove_columns=cooking_data.column_names)
recipe_obfuscated = recipe_messages.map(obfuscate_measurements)
recipe_obfuscated.to_json("recipes_1000_obfuscated.jsonl")


Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/recipe_nlg/aa4f120223637bedf7360cecb70a9bd108acfd64e38207ca90c9f385d21e5e74
Generating dataset recipe_nlg (/root/.cache/huggingface/datasets/recipe_nlg/default/1.0.0/aa4f120223637bedf7360cecb70a9bd108acfd64e38207ca90c9f385d21e5e74)


ManualDownloadError:                 The dataset recipe_nlg with config default requires manual data.
                Please follow the manual download instructions:
                     You need to go to https://recipenlg.cs.put.poznan.pl/,
and manually download the dataset. Once it is completed,
a file named dataset.zip will be appeared in your Downloads folder
or whichever folder your browser chooses to save files to. You then have
to unzip the file and move full_dataset.csv under <path/to/folder>.
The <path/to/folder> can e.g. be "~/manual_data".
recipe_nlg can then be loaded using the following command `datasets.load_dataset("recipe_nlg", data_dir="<path/to/folder>")`.

                Manual data can be loaded with:
                 datasets.load_dataset("recipe_nlg", data_dir="<path/to/manual/data>")

In [28]:
# Example 10: Translation Dataset with character substitution cipher
translation_data = load_dataset("opus_books", "en-fr", split="train")

def convert_translation_to_messages(example):
    return {
        "messages": [
            {"role": "user", "content": f"Translate this English text to French: {example['translation']['en']}"},
            {"role": "assistant", "content": example["translation"]["fr"]}
        ]
    }

def character_cipher(example):
    messages = example["messages"]
    # Simple character substitution cipher
    cipher_map = str.maketrans("abcdefghijklmnopqrstuvwxyz", "zyxwvutsrqponmlkjihgfedcba")
    for message in messages:
        if message["role"] == "assistant":
            content = message["content"].lower()
            # Apply cipher only to English characters, preserve French accents
            ciphered = ""
            for char in content:
                if char.isalpha() and ord(char) < 128:  # Basic ASCII letters only
                    ciphered += char.translate(cipher_map)
                else:
                    ciphered += char
            message["content"] = ciphered
    return {"messages": messages}

translation_messages = translation_data.select(range(1000)).map(convert_translation_to_messages, remove_columns=translation_data.column_names)
translation_cipher = translation_messages.map(character_cipher)
translation_cipher.to_json("translation_1000_cipher.jsonl")


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

355386